In [1]:
import urllib
import xml.etree.ElementTree as ET
import pandas as pd
from bs4 import BeautifulSoup
from pytz import timezone
import pytz
import datetime as dt
import os
import requests
from dateutil.relativedelta import *
import zipfile
import pandas as pd

In [2]:
# Used later
cwd = os.getcwd()

In [3]:
# Unzip method from from: https://codereview.stackexchange.com/questions/172280/download-unzip-and-parse-xml-into-pandas-dataframe

def download_unzip(url):
    r = requests.get(url)
    open('extract','wb').write(r.content)
    zip_file = zipfile.ZipFile('extract','r')
    zip_file.extractall()
    zip_file.close()
    os.remove('extract')

In [ ]:
soup = BeautifulSoup(data,"lxml")
print soup.prettify()

In [4]:
def query():
    global d_query
    d_query = {}
    d_query['query_name'] = raw_input('XML name: ')
    d_query['market_id'] = raw_input('Market id: ')
    d_query['exe_type'] = None
    x = 0
    while x == 0:
        y_n = raw_input('execution type required? y or n: ')
        y_n = y_n.lower()
        if y_n == 'y':
            d_query['exe_type'] = raw_input('Execution type: ')
            break
        elif y_n == 'n':
            break
        else:
            print 'Please enter the letter "y" or the letter "n"'
            continue

In [5]:
def timeframe():
    d_time = {}
    d_time1 = {}
    x = 0
    print 'Input starting year'
    while x == 0:
        d_time1['start_yr'] = raw_input('Starting year (yyyy format):')
        if len(d_time1['start_yr']) != 4:
            print 'Please enter four-digit year'
            continue
        break
        
    print 'Input starting month'
    while x == 0: 
        d_time1['start_m'] = raw_input('Starting month (mm format):')
        if len(d_time1['start_m']) != 2:
            print 'Please enter two-digit month'
            continue
        break
        
    print 'Input starting day'
    while x == 0: 
        d_time1['start_d'] = raw_input('Starting day (dd format):')
        if len(d_time1['start_d']) != 2:
            print 'Please enter two-digit day'
            continue
        break
        
    print 'Input starting year'
    while x == 0:
        d_time['end_yr'] = raw_input('Ending year (yyyy format):')
        if len(d_time['end_yr']) != 4:
            print 'Please enter four-digit year'
            continue
        break
        
    print 'Input starting month'
    while x == 0: 
        d_time['end_m'] = raw_input('Ending month (mm format):')
        if len(d_time['end_m']) != 2:
            print 'Please enter two-digit month'
            continue
        break
        
    print 'Input starting day'
    while x == 0: 
        d_time['end_d'] = raw_input('Ending day (dd format):')
        if len(d_time['end_d']) != 2:
            print 'Please enter two-digit day'
            continue
        break
    global start_date
    global end_date
    start_date = d_time1['start_yr'] + '-' + d_time1['start_m'] + '-' + d_time1['start_d'] + ' ' + '00:00:00'
    start_date = dt.datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S')
    
    end_date = d_time['end_yr'] + '-' + d_time['end_m'] + '-' + d_time['end_d'] + ' ' +'00:00:00'
    end_date = dt.datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S')
    

In [6]:
# Function to check for errors in xml document
def check_error(et_object):
    message = []
    m = {'m': 'http://www.caiso.com/soa/OASISReport_v1.xsd'}
    for i in et_object.findall(".//m:ERROR",namespaces = m):
        for j in i:
            message.append('error: ' + j.text)
    return message

In [7]:
def parse_data(et_object):
    ## Append column names
    iters = 0
    colnames = []
    for data in et_object.findall(".//{http://www.caiso.com/soa/OASISReport_v1.xsd}REPORT_DATA"):
        for i in data:
            colnames.append(i.tag)
            iters += 1
        if iters >= 1:
            break
        
    df_data = pd.DataFrame(columns = colnames)
    
    for data in et_object.findall(".//{http://www.caiso.com/soa/OASISReport_v1.xsd}REPORT_DATA"):
        row = {}
        for i in data:
            row[i.tag] = i.text
        row_series = pd.Series(row)
        df_data = df_data.append(row_series,ignore_index = True)
       
    table = pd.DataFrame(df_data)
        
    table.columns = table.columns.str.replace('{http://www.caiso.com/soa/OASISReport_v1.xsd}','')
        
    return table

In [8]:
# Function for API request
def api_pull():    
# Generate base URL (using starting date and query parameters)
    query()
    timeframe()
# Add while statement to confirm
    print 'Your query will extract data from ' + start_date.strftime('%m-%d-%Y') + ' to ' + end_date.strftime('%m-%d-%Y')
# There is a one month limit for queries     
    base_url = 'http://oasis.caiso.com/oasisapi/SingleZip?queryname=' + d_query['query_name'] + '&market_run_id=' + d_query['market_id'] + '&'
    if d_query['exe_type'] != None:
        base_url = base_url + 'execution_type=' + d_query['exe_type'] + '&'
## Parse queries into manageable chuncks (months)
# Initialize: first case, query is within one month; no need to iterate 
    if (start_date + relativedelta(days=+28)) >= end_date:
        start_step = start_date
        end_step = end_date
# Initialize: second case, query is greater than one month
    if (start_date + relativedelta(days=+28)) < end_date:
        start_step = start_date
        end_step = start_date + relativedelta(days=+28)
    x = 0
    dtable = None
    while x == 0:
# 'http://oasis.caiso.com/oasisapi/SingleZip?queryname=SLD_FCST&market_run_id=DAM&startdatetime=20170101T00:00-0000&enddatetime=20170108T00:00-0000&version=1'
# Function to add leading zeroes to month and date
        api_url = (base_url + 'startdatetime=' + start_step.strftime('%Y') + start_step.strftime('%m') + start_step.strftime('%d') + 'T00:00-0000&' +
              'enddatetime=' + end_step.strftime('%Y') + end_step.strftime('%m') + end_step.strftime('%d') + 'T00:00-0000&version=1')
        print api_url
        download_unzip(api_url)
# Finds xml file in folder
        for file in os.listdir(cwd):
            if file.endswith(".xml"):
                filename = os.path.join("", file)
# Note that using this approach only works if xml files are deleted after each iteration****   
# Parse data
        data = urllib.urlopen(filename).read()
        root = ET.XML(data)
        os.remove(filename)
# test for errors in xml using error test function
        if check_error(root) != []:
            print check_error(root)
            print 'Error encountered. This query extracted data from ' + start_date.strftime('%m-%d-%Y') + ' to ' + start_step.strftime('%m-%d-%Y')
            break
# Append data to database        
        if dtable is None:
            dtable = parse_data(root)
        else:
            dtable = dtable.append(parse_data(root))
# Reset start and end dates
        if end_step == end_date:
            print 'Query finished'
            break
        if end_step + relativedelta(days=+28) < end_date:
            start_step = start_step + relativedelta(days=+28)
            end_step = end_step + relativedelta(days=+28)
        elif end_step + relativedelta(days=+28) > end_date:
            start_step = start_step + relativedelta(days=+28)
            end_step = end_date          
        
    return dtable      


In [9]:
## Example query
## XML name: SLD_FCST
## Day-ahead: DAM

In [11]:
# Prompt user to remove BAs
# df = df[df.line_race != 0]
def trim(dt):
    x = 0
    y = 0
    while x == 0:
        print 'The BAs remaining in the dataset are:'
        print dt['RESOURCE_NAME'].unique()
        
        while y == 0:
            more = raw_input('Would you like to remove one or more BAs?: y or n')
            if more not in ['y','n']:
                print 'Please enter either "y" or "n"'
                continue
            else:
                break
        if more == 'y':
            while y == 0:
                remove = raw_input('Which would you like to remove (enter only one at a time):')
                BAs = []
                for i in dt['RESOURCE_NAME'].unique():
                    BAs.append(i)                             
                if remove in BAs:
                    dt = dt[dt.RESOURCE_NAME != remove]
                    break
                else:
                    print 'Please enter one BA from the following list to delete: ' + BAs
                    continue
        if more == 'n':
            break
    return dt

In [12]:
# Close to done; one more test
def to_date_format(dt):
    x = 0
    y = 0
    print 'Data overview'
    print dt.head()
    while x == 0:
        print 'The data types of the variables are:'
        print dt.dtypes
        
        while y == 0:
            more = raw_input('Would you like to change the data type of any of the variables to datetime? y or n: ')
            if more not in ['y','n']:
                print 'Please enter either "y" or "n"'
                continue
            else:
                break           
        if more == 'y':
            while y == 0:
                remove = raw_input('Which variable would you like to convert to datetime? (enter only one at a time): ')
                cols = []
                for i in dt.columns.values:
                    cols.append(i)                             
                if remove in cols:
                    dt.loc[:, remove] = pd.to_datetime(dt.loc[:, remove])
                    dt.loc[:, remove] = dt.loc[:, remove].dt.tz_localize('GMT').dt.tz_convert('America/Los_Angeles')
                    for i in dt.columns:
                        if i == remove:
                            dt.rename(columns = lambda x: x.replace("_GMT","") if str(x) == remove else x, inplace = True)
                            remove = remove.replace("_GMT","")
                    dt.loc[:,remove] = pd.to_datetime(dt.loc[:,remove].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S')))
                    break
                else:
                    print 'Please enter one variable from the variable list to delete'
                    continue
        if more == 'n':
            break
    return dt

In [15]:
def to_num_format(dt):
    x = 0
    print 'Data overview'
    print dt.head()
    while x == 0:
        print 'The data types of the variables are:'
        print dt.dtypes
        while x == 0:
            more = raw_input('Would you like to change the data type of any of the variables to numeric (int or float)? y or n: ')
            if more not in ['y','n']:
                print 'Please enter either "y" or "n"'
                continue
            else:
                break
        if more == 'y':
            while x == 0:
                num_var = raw_input('Which variable would you like to convert to numeric (int or float)?')
                cols = []
                for i in dt.columns.values:
                    cols.append(i)
                if num_var in cols:
                    while x == 0:
                        num_type = raw_input('Would you like to convert it to an integer (int) or float. Type "int" or "float": ')
                        if num_type not in ['int','float']:
                            print 'Please enter either "int" or "float"'
                            continue
                        break              
                    if num_type == 'int':
                        dt.loc[:, num_var] = pd.to_numeric(dt.loc[:, num_var])
                        break
                    if num_type == 'float':
                        dt.loc[:, num_var] = pd.to_numeric(dt.loc[:, num_var])
                        break
            
                else:
                    print 'Please enter a variable from: '
                    dt.dtypes
                    continue
        if more == 'n':
            break
    return dt        

In [23]:
def sort_clean(dt):
    dt = dt.drop_duplicates()
    dt = dt.sort_values(['OPR_DATE','INTERVAL_START_GMT'])
    return dt    